# **CS 598 Deep Learning for Healthcare Final Project**

# **Domain Knowledge Guided Deep Learning with Electronic Health Records**



Team ID: 74

Paper ID: 69


---


Byunggeun BK Park (bpark14@illinois.edu)

Spencer Arbour (sarbour2@illinois.edu)

Yun Gao (yungao3@illinois.edu)


# Introduction

Please find our public repository at: https://drive.google.com/drive/folders/1SOuU9TfnHdI9R6mxhpTNvlaZ61zbgn5p?usp=sharing

Public Github repository: https://github.com/BK1147/CS598_DLH_Final_Project.git

Youtube or GDrive:




---



In the realm of healthcare, the utilization of Electronic Health Records (EHRs) for predictive analytics stands as a beacon of opportunity for enhancing patient care and resource allocation. However, the complexity inherent in EHR data, characterized by its high dimensionality, sparsity, and irregularity, poses formidable challenges to accurate prediction of clinical outcomes. Addressing these challenges is paramount for unlocking the full potential of EHRs in revolutionizing healthcare practices.

This paper presents a pioneering solution to the intricate problem of clinical risk prediction using EHRs: the Domain Knowledge Guided Recurrent Neural Networks (DG-RNN). By seamlessly integrating rich medical knowledge into the predictive modeling process, DG-RNN transcends the limitations of conventional approaches, offering unprecedented accuracy, interpretability, and adaptability in clinical risk assessment.

The cornerstone of DG-RNN lies in its innovative architecture, which harnesses the power of deep learning alongside domain-specific medical insights. Through a Graph-Based Attention Mechanism, the model dynamically incorporates complex relationships between medical events, treatments, and outcomes, enriching the predictive process with invaluable contextual information. Furthermore, a Global Max-Pooling Layer enhances interpretability by spotlighting the most influential medical events contributing to each prediction, thereby bridging the gap between data-driven predictions and clinical intuition.

Key to the efficacy of DG-RNN is its adept handling of the idiosyncrasies of EHR data. By encoding temporal irregularities and leveraging advanced attention mechanisms, the model navigates the sparsity and irregularity of patient records with finesse, ensuring robust performance across diverse datasets and prediction tasks.

Through rigorous evaluation on real-world EHR datasets, particularly focusing on heart failure risk prediction, DG-RNN showcases its superiority over traditional methods and contemporary deep learning approaches. Not only does it excel in accuracy, but it also stands out for its interpretability, aligning seamlessly with clinical insights and facilitating informed decision-making by healthcare practitioners.


---



# Scope of Reproducibility

The original paper introduces the Domain Knowledge Guided Recurrent Neural Networks (DG-RNN), a model that incorporates complex medical knowledge directly into the predictive modeling process using a recurrent neural network architecture. This model is equipped with a graph-based attention mechanism and a global max-pooling layer, designed to enhance predictive accuracy and interpretability for clinical outcomes. The study investigates four variations of the RNN model:

1. RNN without additional layers
2. RNN with reversed input layer
3. RNN with global max-pooling
4. RNN with global max-pooling + reversed input layer

These models are evaluated against traditional machine learning methods such as Random Forest, Logistic Regression, and Support Vector Machine, as well as advanced deep learning models including GRU, LSTM, and RETAIN. Additionally, for the ablation study, we include a naive bi-directional RNN model with an additional reversed input layer to compare with other models.

Due to the incomplete availability of the original implementation files and resources, we plan to recreate the high-level concepts and methodologies presented in the original paper. Our experimental setup will use the MIMIC-IV dataset to validate the effectiveness and reproducibility of the proposed methods.

#### Hypotheses to be Tested

- **Hypothesis 1**: The basic RNN model will outperform traditional machine learning methods (Random Forest, Logistic Regression, Support Vector Machine) in predicting patient risk. This is expected due to the RNN's ability to effectively capture temporal dependencies and sequential patterns in EHR data, which are critical for accurate clinical risk prediction.

- **Hypothesis 2**: An RNN model enhanced with reversed input layer and a global max-pooling layer will exhibit superior performance compared to a basic RNN configuration. The reversed input layer is anticipated to improve the model’s ability to dynamically incorporate relevant medical knowledge, while the global max-pooling layer should help in capturing the most salient features from the input sequences, thus enhancing predictive accuracy and model generalization.

By testing these hypotheses, we aim to assess the reproducibility of the original paper's claims and explore the practical benefits of advanced RNN architectures in clinical risk prediction using MIMIC-IV data. Through this process, we also plan to identify any potential challenges and limitations in replicating the study’s results, which will contribute valuable insights for future research in the field of healthcare analytics.






# Methodology


## Environment

Python version
*  Python 3.11
    
Dependencies/packages needed
*  Refer to requirements.txt file




##  Data

### Data download instruction
*  An encrypted version of the patient used for running our model have been include `enc_hfs_dataset_pkl`.
*  A secret key will be included as a separate download/note in our submission. Save this file to the root directory for this project as `secret.key`.
*  Alternatively, create a text file using Vim (or similar) to copy/paste the key provided, saving the file as `secret.key` in the root directory.

### Data description
*  MIMIC IV data tables (version 2.2) are used as a basis for patient data analysis. The data was collected from PhysioNet. Our dataset was pulled from 'hosp' patient EHRs, and were filtered to only contain ICD-9 codes.



---

Pre-processing of the MIMIC IV dataset has been moved into `Data_Preprocess.ipynb`. Please reference that notebook for questions regarding filtering of the original dataset.

In [1]:
import os
import sys
import pickle
import random
import numpy as np
import pandas as pd
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# ensure data path exists in directory
DATA_PATH = 'data/'
DATA_PATH_PICKLE = 'data/'
os.makedirs(os.path.dirname(DATA_PATH), exist_ok=True)

In [79]:
# Cleans up file structure from previous runs (if any)
dir = DATA_PATH

for file in os.listdir(dir):
    file_path = os.path.join(dir, file)
    os.remove(file_path)

if os.path.isfile('enc_diagnoses'):
    os.remove('enc_diagnoses')

##### Unencrypt `enc_hfs_dataset_pkl` and store in `data` directory

In [9]:
from cryptography.fernet import Fernet

with open('secret.key', 'rb') as mykey:
    key = mykey.read()

f = Fernet(key)

with open('enc_hfs_dataset_pkl', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open(os.path.join(DATA_PATH,'hfs_dataset_pkl.zip'), 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

In [3]:
# Unzip pickle files and store in /data directory
import shutil
import pathlib

shutil.unpack_archive(os.path.join(DATA_PATH,'hfs_dataset_pkl.zip'), os.path.join(DATA_PATH,'hfs_pickle_files/'), format='zip')

dir = os.path.join(DATA_PATH, 'hfs_pickle_files/')

for file in os.listdir(dir):
    file_path = os.path.join(dir, file)
    shutil.move(file_path, DATA_PATH)

pathlib.Path.unlink(os.path.join(DATA_PATH, 'hfs_dataset_pkl.zip'))
pathlib.Path.rmdir(os.path.join(DATA_PATH, 'hfs_pickle_files'))

In [6]:
# comment for the each data
pids = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'pids.pickle'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'vids.pickle'), 'rb'))
hfs = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'hfs.pickle'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'seqs.pickle'), 'rb'))
types = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'types.pickle'), 'rb'))
rtypes = pickle.load(open(os.path.join(DATA_PATH_PICKLE,'rtypes.pickle'), 'rb'))

In [7]:
len(pids)

36975

### Pre-processing: Filter 'patient_df' and 'diagnoses_df' into list data used for RNN

where
* pids: contains the patient ids
* vids: contains a list of visit ids for each patient
* hfs: contains the heart failure label (0: normal, 1: heart failure) for each patient
* seqs: contains a list of visit (in ICD9 codes) for each patient
* types: contains the map from ICD9 codes to ICD-9 labels
* rtypes: contains the map from ICD9 labels to ICD9 codes

Read in `d_icd_diagnoses.csv` and `diagnoses.csv` as Pandas DataFrames

In [10]:
with open('enc_d_icd_diagnoses', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open(os.path.join(DATA_PATH,'d_icd_diagnoses.csv'), 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

In [11]:
d_icd_diagnoses_df = pd.read_csv(DATA_PATH + '/d_icd_diagnoses.csv', sep=',', header='infer')
print(d_icd_diagnoses_df.head(5))

  icd_code  icd_version                             long_title
0     0010            9         Cholera due to vibrio cholerae
1     0011            9  Cholera due to vibrio cholerae el tor
2     0019            9                   Cholera, unspecified
3     0020            9                          Typhoid fever
4     0021            9                    Paratyphoid fever A


In [8]:
import gdown

url = "https://drive.google.com/file/d/1sZIXoSLOnC0W6D5DxQFXmsJozBO_gcGX/view?usp=share_link"
output = 'enc_diagnoses'

gdown.download(url=url, output=output, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1sZIXoSLOnC0W6D5DxQFXmsJozBO_gcGX
From (redirected): https://drive.google.com/uc?id=1sZIXoSLOnC0W6D5DxQFXmsJozBO_gcGX&confirm=t&uuid=039efb4a-07ec-440a-af10-1c375ce831c7
To: /Users/spenny/CS598_Final_Proj/CS598_DLH_Final_Project/enc_diagnoses
100%|██████████| 180M/180M [00:19<00:00, 9.37MB/s] 


'enc_diagnoses'

In [9]:
with open('enc_diagnoses', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open(os.path.join(DATA_PATH,'diagnoses_icd.csv'), 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

In [10]:
diagnoses_df = pd.read_csv(DATA_PATH + '/diagnoses_icd.csv', sep=',', header='infer')
print(diagnoses_df.head(5))

   subject_id   hadm_id  seq_num icd_code  icd_version
0    10000032  22595853        1     5723            9
1    10000032  22595853        2    78959            9
2    10000032  22595853        3     5715            9
3    10000032  22595853        4    07070            9
4    10000032  22595853        5      496            9




---



The MIMIC-IV dataset is a large and comprehensive collection of medical records, containing over 120,000 unique patient identifiers ('pid'). While this extensive dataset provides a wealth of information, training machine learning models on such a vast dataset can lead to overtraining and overfitting issues. Overfitting occurs when a model learns the training data too well, including its noise and outliers, thereby reducing its ability to generalize to new, unseen data.

To mitigate these challenges and improve the model's performance, we have decided to train our models on a smaller, carefully selected subset of the MIMIC-IV dataset. By focusing on a more manageable dataset, we aim to create models that are more robust, generalize better to new data, and ultimately yield more reliable predictions in clinical settings.

In [12]:
total_length = len(pids)
random_indices = random.sample(range(total_length), 1000)

# Select the 2000 elements using the random indices
pids = [pids[i] for i in random_indices]
vids = [vids[i] for i in random_indices]
hfs = [hfs[i] for i in random_indices]
seqs = [seqs[i] for i in random_indices]

Pre-Processing for Machine Learning model: Random Forest (RF), Logistic Regression(LR), Support Vector Machine (SVM)

In [2]:
d_icd_diagnoses_df = pd.read_csv(DATA_PATH + '/d_icd_diagnoses.csv', sep=',', header='infer')
diagnoses_df = pd.read_csv(DATA_PATH + '/diagnoses_icd.csv', sep=',', header='infer')

merged_df = pd.merge(diagnoses_df,d_icd_diagnoses_df, on=['icd_code', 'icd_version'], how='left')
merged_df['hfs'] = merged_df['long_title'].str.contains("heart failure").astype(int)
merged_df = merged_df[merged_df['icd_version'] == 9]

In [3]:
grouped_df = merged_df.groupby('subject_id').agg({
    'seq_num': 'sum',
    'hfs': 'max'
}).reset_index()

grouped_df = grouped_df[:1000]

##  Model

Model architecture

---

**RNN without additional layers**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
Linear (fully connected) | input_size=hidden_size , output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)

**RNN with reversed input layer**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
GRU (Reverse RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
Linear (fully connected) | input_size=hidden_size * 2, output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)

**RNN with global max-pooling layer**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
AdaptiveMaxPool1d | output_size=hidden_size | - | (batch_size, hidden_size)
Linear (fully connected) | input_size=hidden_size, output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)

**RNN with global max-pooling layer + reversed input layer**

Layers | Configuration | Activation Function | Output Dimension (batch, feature)
--- | --- | --- | ---
Embedding | num_embeddings=num_codes, embedding_dim=128 | - | (batch_size, seq_len, embedding_dim)
GRU (RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
GRU (Reverse RNN) | input_size=128, hidden_size=hidden_size | - | (batch_size, seq_len, hidden_size)
AdaptiveMaxPool1d | output_size=hidden_size | - | (batch_size, hidden_size)
Linear (fully connected) | input_size=hidden_size * 2, output_size=1 | - | (batch_size, 1)
Sigmoid | - | - | (batch_size, 1)


Training objectives:
*   Loss Function: Binary Cross-Entropy Loss (assuming the model is for binary classification since the output is passed through a sigmoid activation).
*   Optimizer: Adam optimizer with a learning rate of 1e-3.




## Disclamer

Disclaimer: We utilized the basic RNN structure code from Homework 3 for the RNN model. Using this basic structure as a foundation, we will combine it with a naive bi-directional mechanism and a global max-pooling layer.

In [13]:
# Custom dataset
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import DataLoader

class CustomDataset(Dataset):
  def __init__(self, seqs, hfs):
    self.seqs = seqs
    self.hfs = hfs

  def __len__(self):
    return len(self.seqs)

  def __getitem__(self, index):
    return self.seqs[index], self.hfs[index]

dataset = CustomDataset(seqs, hfs)

In [14]:
def collate_fn(data):
    sequences, labels = zip(*data)
    y = torch.tensor(labels, dtype=torch.float)

    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)

    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            padded_visit = torch.tensor(visit + [0] * (max_num_codes - len(visit)), dtype=torch.long)
            x[i_patient, j_visit, :] = padded_visit
            masks[i_patient, j_visit, :] = padded_visit != 0

    for i_patient, patient in enumerate(sequences):
        idx_all_real_visits = torch.sum(x[i_patient, :, :], dim=1) != 0
        idx_padded_visits = torch.sum(x[i_patient, :, :], dim=1) == 0
        reversed_real_visits = torch.flip(x[i_patient, idx_all_real_visits, :], dims=(0,))
        rev_x[i_patient, :, :] = torch.cat((reversed_real_visits, x[i_patient, idx_padded_visits, :]), dim=0)
        rev_masks[i_patient, :, :] = rev_x[i_patient, :, :] != 0

    return x, masks, rev_x, rev_masks, y

def load_data(train_dataset, val_dataset, collate_fn):
    batch_size = 32

    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

    return train_loader, val_loader

def sum_embeddings_with_mask(x, masks):

    masked_embeddings = x * masks.unsqueeze(-1)
    sum_embeddings = masked_embeddings.sum(dim=2)

    return sum_embeddings

def get_last_visit(hidden_states, masks):

    true_visit_length = torch.sum(masks, dim=1)
    idx_last_visit = true_visit_length[:,0] - 1
    last_hidden_state = hidden_states[torch.arange(hidden_states.size(0)), idx_last_visit, :]

    return last_hidden_state

In [15]:
split = int(len(dataset)*0.8)

lengths = [split, len(dataset) - split]
train_dataset, val_dataset = random_split(dataset, lengths)

train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)
print("Length of train dataset:", len(train_dataset))
print("Length of train with HF:", np.sum([element[-1] for element in train_dataset]))
print("Length of train with survive patients:", (len(train_dataset) - np.sum([element[-1] for element in train_dataset])))
print("Length of val dataset:", len(val_dataset))
print("Length of val with HF:", np.sum([element[-1] for element in val_dataset]))
print("Length of val with survive patients:", (len(train_dataset) - np.sum([element[-1] for element in val_dataset])))


Length of train dataset: 800
Length of train with HF: 298
Length of train with survive patients: 502
Length of val dataset: 200
Length of val with HF: 77
Length of val with survive patients: 723


RNN without additional layers

In [16]:
class RNN(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):

        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        logits = self.fc(true_h_n)
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

RNN with reversed input layer

In [17]:
class RNN_Rev(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.rev_rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.fc = nn.Linear(in_features=hidden_size * 2, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):

        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        true_h_n_rev = None

        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, masks)

        rev_output, _ = self.rnn(rev_x)
        true_h_n_rev = get_last_visit(rev_output, masks)

        logits = self.fc(torch.cat([true_h_n, true_h_n_rev], 1))
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

RNN with global max-pooling

In [18]:
class RNN_Pool(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.maxpooling = nn.AdaptiveMaxPool1d(hidden_size)  # Add AdaptiveMaxPool1d layer

        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, masks, rev_x, rev_masks):
        # x, masks, y, adj_matrices
        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        x_max = self.maxpooling(true_h_n.unsqueeze(2).permute(0, 2, 1)).squeeze(2)

        logits = self.fc(x_max) # for maxpooling layer
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

RNN with global max-pooling + reversed input layer

In [19]:
class RNN_Rev_Pool(nn.Module):

    def __init__(self, num_codes, learning_rate = 0.01, hidden_size = 128, dropout = 0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)

        self.rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)
        self.rev_rnn = nn.GRU(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0)

        self.maxpooling = nn.AdaptiveMaxPool1d(hidden_size)  # Add AdaptiveMaxPool1d layer

        self.fc = nn.Linear(in_features=hidden_size * 2, out_features=1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x, masks, rev_x, rev_masks):

        batch_size = x.shape[0]

        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.rnn(x)
        true_h_n = get_last_visit(output, masks)

        true_h_n_rev = None
        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, masks)

        rev_output, _ = self.rnn(rev_x)
        true_h_n_rev = get_last_visit(rev_output, masks)

        true_h_n_max = self.maxpooling(true_h_n.unsqueeze(2).permute(0, 2, 1)).squeeze(2)
        true_h_n_rev_max = self.maxpooling(true_h_n_rev.unsqueeze(2).permute(0, 2, 1)).squeeze(2)

        logits = self.fc(torch.cat([true_h_n_max, true_h_n_rev_max], 2)) 
        probs = self.sigmoid(logits)

        return probs.view(batch_size)

In [20]:
# Loss and Optimizer
criterion = nn.BCELoss()

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Training

Hyperparams

* 

Computational requirements

*   This code ran success fully on a system with the following specifications:
    * Processor: 2.6 GHz Quad-Core intel Core i7
    * Memory: 16 GH 2133 MHz LPDDR3
    * Gradeon Pro 450 2GB Intel HD Graphics 530 1536MB
*   Describe time.... below



In [22]:
from sklearn.metrics import *
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def eval_model(model, val_loader):
    model.to(device).eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        #x, masks, rev_x, rev_masks, y = x.to(device), masks.to(device), rev_x.to(device), rev_masks.to(device), y.to(device)

        y_hat = model(x, masks, rev_x, rev_masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()

        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)

    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)

    return p, r, f, roc_auc

In [23]:
from sklearn.model_selection import ParameterGrid

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'hidden_size': [64, 128, 256],
    'dropout': [0.0, 0.1]
}

def train_with_hyperparameter_tuning(model, train_loader, val_loader, n_epochs, param_grid):
    best_roc_auc_score = 0
    best_params = None

    # Loop over hyperparameter combinations
    for params in ParameterGrid(param_grid):
        print("Training with hyperparameters:", params)

        # Initialize the model with current hyperparameters
        current_model = model(num_codes=len(types), **params).to(device)
        optimizer = torch.optim.Adam(current_model.parameters(), lr=params['learning_rate'])
        print(f"current_model: {current_model}")
        # Training loop
        for epoch in range(n_epochs):
            current_model.train()
            train_loss = 0
            for x, masks, rev_x, rev_masks, y in train_loader:
                optimizer.zero_grad()
                y_hat = current_model(x, masks, rev_x, rev_masks)
                y_hat = y_hat.view(y_hat.shape[0])
                loss = criterion(y_hat, y)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

            train_loss = train_loss / len(train_loader)
            print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))

        # Evaluate on validation set
        _, _, _, roc_auc = eval_model(current_model, val_loader)
        print("Validation roc_auc with current hyperparameters:", roc_auc)

        # Check if this set of hyperparameters is the best so far
        if roc_auc > best_roc_auc_score:
            best_roc_auc_score = roc_auc
            best_params = params

    print("Best hyperparameters:", best_params)
    print("Best roc_auc score:", best_roc_auc_score)

    # Train the model with the best hyperparameters on the full training set
    best_model = model(num_codes=len(types), **best_params).to(device)
    optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params['learning_rate'])
    for epoch in range(n_epochs):
        best_model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            optimizer.zero_grad()
            y_hat = best_model(x, masks, rev_x, rev_masks)
            y_hat = y_hat.view(y_hat.shape[0])
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))

    return best_model

In [24]:
# Call the training function with hyperparameter tuning - RNN without additional layers
best_model_RNN = train_with_hyperparameter_tuning(RNN, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.638951
Epoch: 2 	 Training Loss: 0.472035
Epoch: 3 	 Training Loss: 0.334809
Epoch: 4 	 Training Loss: 0.224091
Epoch: 5 	 Training Loss: 0.144595
Validation roc_auc with current hyperparameters: 0.937493400908035
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.418942
Epoch: 2 	 Training Loss: 0.077670
Epoch: 3 	 Training Loss: 0.013355
Epoch: 4 	 Training Loss: 0.003987
Epoch: 5 	 Training Loss: 0.001999
Validation roc_auc with current hyperparamet

In [25]:
# Call the training function with hyperparameter tuning - RNN with a reversed input layer
best_model_Rev = train_with_hyperparameter_tuning(RNN_Rev, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN_Rev(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.634741
Epoch: 2 	 Training Loss: 0.435689
Epoch: 3 	 Training Loss: 0.287276
Epoch: 4 	 Training Loss: 0.178957
Epoch: 5 	 Training Loss: 0.109499
Validation roc_auc with current hyperparameters: 0.9679020166825044
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN_Rev(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.362468
Epoch: 2 	 Training Loss: 0.041063
Epoch: 3 	 Training Loss: 0.006959
Epoch: 4 	 Trai

In [26]:
# Call the training function with hyperparameter tuning - RNN with global max-pooling
best_model_pool = train_with_hyperparameter_tuning(RNN_Pool, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN_Pool(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.641524
Epoch: 2 	 Training Loss: 0.457636
Epoch: 3 	 Training Loss: 0.323237
Epoch: 4 	 Training Loss: 0.214128
Epoch: 5 	 Training Loss: 0.137468
Validation roc_auc with current hyperparameters: 0.9571322985957132
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN_Pool(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.427149
Epoch: 2 	 Training Loss: 0.060968
Epoch: 3 	 Training Loss: 0.008834
Epo

In [27]:
# Call the training function with hyperparameter  - RNN with global max-pooling + reversed input layer
best_model_rev_pool = train_with_hyperparameter_tuning(RNN_Rev_Pool, train_loader, val_loader, n_epochs=5, param_grid=param_grid)

Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001}
current_model: RNN_Rev_Pool(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	 Training Loss: 0.615994
Epoch: 2 	 Training Loss: 0.410773
Epoch: 3 	 Training Loss: 0.256110
Epoch: 4 	 Training Loss: 0.149673
Epoch: 5 	 Training Loss: 0.088141
Validation roc_auc with current hyperparameters: 0.9747650723260479
Training with hyperparameters: {'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.01}
current_model: RNN_Rev_Pool(
  (embedding): Embedding(7328, 128)
  (rnn): GRU(128, 64, batch_first=True)
  (rev_rnn): GRU(128, 64, batch_first=True)
  (maxpooling): AdaptiveMaxPool1d(output_size=64)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Epoch: 1 	

Evaluation

* RNN without additional layers
* RNN with a reversed input layer
* RNN with global max-pooling
* RNN with global max-pooling + reversed input layer

In [32]:
# RNN without additional layers
p, r, f, roc_auc  = eval_model(best_model_RNN, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.9474
Recall: 0.9351
F1-score: 0.9412
ROC AUC score: 0.9692


In [33]:
# RNN with a reversed input layer
p, r, f, roc_auc  = eval_model(best_model_Rev, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.9494
Recall: 0.9740
F1-score: 0.9615
ROC AUC score: 0.9860


In [34]:
# RNN with global max-pooling
p, r, f, roc_auc  = eval_model(best_model_pool, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.9375
Recall: 0.9740
F1-score: 0.9554
ROC AUC score: 0.9758


In [35]:
# RNN with global max-pooling + reversed input layer
p, r, f, roc_auc  = eval_model(best_model_rev_pool, val_loader)
print(f'Precision: {p:.4f}')
print(f'Recall: {r:.4f}')
print(f'F1-score: {f:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.9351
Recall: 0.9351
F1-score: 0.9351
ROC AUC score: 0.9844


### Training/Testing the Machine Learning Model: Random Forest (RF), Logistic Regression(LR), Support Vector Machine (SVM)

Based on the paper, they computed the counts of each medical event for each patient to predict the heart failure outcome.

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

X = grouped_df[['subject_id','seq_num']]
y = grouped_df['hfs']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

Random Forest (RF)

In [37]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(RandomForestClassifier(random_state=16), param_grid, cv=5, scoring='roc_auc')

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_rf_classifier = grid_search.best_estimator_

# Fit the model with best parameters
best_rf_classifier.fit(X_train, y_train)

# Predict using the best model
y_pred = best_rf_classifier.predict(X_test)

# Evaluate the model
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, best_rf_classifier.predict_proba(X_test)[:, 1])
print("Best Parameters:", best_params)
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}
Precision: 0.2667
Recall: 0.1905
F1-score: 0.2222
ROC AUC score: 0.7849


Logistic Regression (LR)

In [38]:
# Define the parameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(random_state=16, max_iter=100), param_grid, cv=5, scoring='roc_auc')

# Perform the grid search
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_logreg = grid_search.best_estimator_

# Fit the model with best parameters
best_logreg.fit(X_train, y_train)

# Predict using the best model
y_pred = best_logreg.predict(X_test)

# Evaluate the model
precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, best_logreg.predict_proba(X_test)[:, 1])
print("Best Parameters:", best_params)
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')


/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warni

Best Parameters: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Precision: 0.4000
Recall: 0.0952
F1-score: 0.1538
ROC AUC score: 0.8500


/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/bk/CS598_DLH_Final_Project/env/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warni

Support Vector Machine (SVM)

In [39]:
svm_classifier = SVC(kernel='linear', C=1, probability=True, random_state=16)

svm_classifier.fit(X_train, y_train)

y_prob = svm_classifier.predict_proba(X_test)[:, 1]

threshold = 0.5  # Adjust this threshold based on the precision-recall trade-off
y_pred = (y_prob > threshold).astype(int)

precision, recall, fscore, support = precision_recall_fscore_support(y_test, y_pred, average='binary')
roc_auc = roc_auc_score(y_test, svm_classifier.predict_proba(X_test)[:, 1])

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {fscore:.4f}')
print(f'ROC AUC score: {roc_auc:.4f}')

Precision: 0.2500
Recall: 0.0476
F1-score: 0.0800
ROC AUC score: 0.7765


### Training/Testing the Deep Learning Model: Gated Recurrent Unit (GRU), Long Short-Term Memory (LSTM), RETAIN


Disclaimer: In the development of the Gated Recurrent Unit (GRU), Long Short-Term Memory (LSTM), and RETAIN models, we leveraged the foundational structure code provided in Homework 3-4 as a starting point. While our implementations may share similarities with the base code, significant modifications and optimizations have been made to adapt them to our specific research objectives and dataset.

In [42]:
assert len(pids) == len(vids) == len(hfs) == len(seqs)

In [43]:
print("number of heart failure patients:", sum(hfs))
print("ratio of heart failure patients: %.2f" % (sum(hfs) / len(hfs)))

number of heart failure patients: 375
ratio of heart failure patients: 0.38


In [37]:
def collate_fn(data):
    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            for k_diag, diag in enumerate(visit):
                x[i_patient][j_visit][k_diag] = diag
                masks[i_patient][j_visit][k_diag] = 1
    
    return x, masks, y

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def eval_model(model, val_loader):
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, y in val_loader:
        y_hat = model(x, masks)
        y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    
    roc_auc = roc_auc_score(y_true, y_score)
    
    return p, r, f, roc_auc

def train(model, train_loader, val_loader, n_epochs, optimizer):
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, y in train_loader:
            loss = None
            optimizer.zero_grad()
            y_hat = model(x, masks)
            y_hat = y_hat.view(y_hat.shape[0])
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval_model(model, val_loader)
        print('Epoch: {} \t Validation p: {:.3f}, r:{:.3f}, f: {:.3f}, roc_auc: {:.3f}'
              .format(epoch+1, p, r, f, roc_auc))

Gated Recurrent Unit (GRU)

In [38]:
class NaiveGRU(nn.Module):
    
    def __init__(self, num_codes):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)
        self.gru = nn.GRU(input_size=128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(in_features=128, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, masks):
        batch_size = x.shape[0]
        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)
        output, _ = self.gru(x)
        true_h_n = get_last_visit(output, masks)
        logits = self.fc(true_h_n)
        probs = self.sigmoid(logits)
        return probs.view(batch_size)

naive_gru = NaiveGRU(num_codes = len(types))

gru_optimizer = torch.optim.Adam(naive_gru.parameters(), lr=0.001)

In [39]:
n_epochs = 5   # number of epochs to train the model

train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)
train(naive_gru, train_loader, val_loader, n_epochs, gru_optimizer)

Epoch: 1 	 Training Loss: 0.576874
Epoch: 1 	 Validation p: 0.793, r:0.844, f: 0.818, roc_auc: 0.922
Epoch: 2 	 Training Loss: 0.319999
Epoch: 2 	 Validation p: 0.833, r:0.909, f: 0.870, roc_auc: 0.952
Epoch: 3 	 Training Loss: 0.173464
Epoch: 3 	 Validation p: 0.885, r:0.896, f: 0.890, roc_auc: 0.959
Epoch: 4 	 Training Loss: 0.088321
Epoch: 4 	 Validation p: 0.911, r:0.935, f: 0.923, roc_auc: 0.964
Epoch: 5 	 Training Loss: 0.048066
Epoch: 5 	 Validation p: 0.910, r:0.922, f: 0.916, roc_auc: 0.965


Long Short-Term Memory (LSTM)

In [40]:
class NaiveLSTM(nn.Module):
    
    def __init__(self, num_codes):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_codes, embedding_dim=128)
        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(in_features=128, out_features=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, masks):

        batch_size = x.shape[0]
        x = self.embedding(x)
        x = sum_embeddings_with_mask(x, masks)

        output, _ = self.lstm(x)
        true_h_n = get_last_visit(output, masks)

        logits = self.fc(true_h_n)
        probs = self.sigmoid(logits)
        return probs.view(batch_size)

naive_lstm = NaiveLSTM(num_codes = len(types))

lstm_optimizer = torch.optim.Adam(naive_lstm.parameters(), lr=0.001)

In [41]:
train(naive_lstm, train_loader, val_loader, n_epochs, lstm_optimizer)

Epoch: 1 	 Training Loss: 0.619597
Epoch: 1 	 Validation p: 0.731, r:0.987, f: 0.840, roc_auc: 0.959
Epoch: 2 	 Training Loss: 0.403011
Epoch: 2 	 Validation p: 0.807, r:0.922, f: 0.861, roc_auc: 0.956
Epoch: 3 	 Training Loss: 0.235102
Epoch: 3 	 Validation p: 0.833, r:0.974, f: 0.898, roc_auc: 0.961
Epoch: 4 	 Training Loss: 0.125985
Epoch: 4 	 Validation p: 0.877, r:0.922, f: 0.899, roc_auc: 0.959
Epoch: 5 	 Training Loss: 0.071157
Epoch: 5 	 Validation p: 0.887, r:0.922, f: 0.904, roc_auc: 0.960


RETAIN

In [56]:
def collate_fn(data):
    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            for k_diag, diag in enumerate(visit):
                x[i_patient][j_visit][k_diag] = diag
                masks[i_patient][j_visit][k_diag] = 1
        
        for j_visit, visit in enumerate(patient):
            rev_x[i_patient][j_visit] = x[i_patient][len(patient) - 1 - j_visit]
            rev_masks[i_patient][j_visit] = masks[i_patient][len(patient) - 1 - j_visit]
    
    return x, masks, rev_x, rev_masks, y

In [68]:
def load_data(train_dataset, val_dataset, collate_fn):
    batch_size = 32
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    return train_loader, val_loader

train_loader, val_loader = load_data(train_dataset, val_dataset, collate_fn)

In [69]:
class AlphaAttention(torch.nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.a_att = nn.Linear(embedding_dim, 1)

    def forward(self, g, rev_masks):
        g = self.a_att(g)
        g[torch.sum(rev_masks, dim = 2) == 0] = -1e9
        m = nn.Softmax(dim=1)
        return m(g)
    
class BetaAttention(torch.nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.b_att = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, h):
        h = self.b_att(h)
        return torch.tanh(h)
    
def attention_sum(alpha, beta, rev_v, rev_masks):
    masks_ = torch.sum(rev_masks, dim = 2)
    masks_[masks_ != 0] = True
    masks_ = masks_.unsqueeze(2)

    return torch.sum(alpha * beta * rev_v * masks_, dim = 1)

def sum_embeddings_with_mask(x, masks):
    x = x * masks.unsqueeze(-1)
    x = torch.sum(x, dim = -2)
    return x

In [70]:
class RETAIN(nn.Module):
    
    def __init__(self, num_codes, embedding_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(num_codes, embedding_dim)
        self.rnn_a = nn.GRU(embedding_dim, embedding_dim, batch_first=True)
        self.rnn_b = nn.GRU(embedding_dim, embedding_dim, batch_first=True)
        self.att_a = AlphaAttention(embedding_dim)
        self.att_b = BetaAttention(embedding_dim)
        self.fc = nn.Linear(embedding_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, masks, rev_x, rev_masks):
        rev_x = self.embedding(rev_x)
        rev_x = sum_embeddings_with_mask(rev_x, rev_masks)
        g, _ = self.rnn_a(rev_x)
        h, _ = self.rnn_b(rev_x)
        alpha = self.att_a(g, rev_masks)
        beta = self.att_b(h)
        c = attention_sum(alpha, beta, rev_x, rev_masks)
        logits = self.fc(c)
        probs = self.sigmoid(logits)
        return probs.squeeze(dim=-1)

In [73]:
def eval(model, val_loader):
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        y_logit = model(x, masks, rev_x, rev_masks)
        y_hat = model(x, masks, rev_x, rev_masks)
        y_hat = (y_hat > 0.5).int()
        y_score = torch.cat((y_score,  y_logit.detach().to('cpu')), dim=0)
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
    return p, r, f, roc_auc
    
def train(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x, masks, rev_x, rev_masks)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval(model, val_loader)
        print('Epoch: {} \t Validation p: {:.3f}, r:{:.3f}, f: {:.3f}, roc_auc: {:.3f}'.format(epoch+1, p, r, f, roc_auc))
    return round(roc_auc, 2)

In [76]:
retain = RETAIN(num_codes = len(types))
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(retain.parameters(), lr=1e-3)

n_epochs = 5
train(retain, train_loader, val_loader, n_epochs)

Epoch: 1 	 Training Loss: 0.588016
Epoch: 1 	 Validation p: 0.643, r:0.935, f: 0.762, roc_auc: 0.936
Epoch: 2 	 Training Loss: 0.263385
Epoch: 2 	 Validation p: 0.880, r:0.857, f: 0.868, roc_auc: 0.957
Epoch: 3 	 Training Loss: 0.122212
Epoch: 3 	 Validation p: 0.934, r:0.922, f: 0.928, roc_auc: 0.967
Epoch: 4 	 Training Loss: 0.061360
Epoch: 4 	 Validation p: 0.945, r:0.896, f: 0.920, roc_auc: 0.965
Epoch: 5 	 Training Loss: 0.037007
Epoch: 5 	 Validation p: 0.958, r:0.896, f: 0.926, roc_auc: 0.966


0.97

# Results

In conclusion, the RNN model outperformed all other machine learning models in terms of Precision, Recall, F1-score, and ROC AUC score, demonstrating its superior predictive performance for the task of predicting patient risk. The Logistic Regression, Random Forest, and SVM models showed varying degrees of performance, with the SVM model suffering from severe class imbalance issues


* Table of results (no need to include additional experiments, but main reproducibility result should be included)

| Model        | Precision  | Recall     | F1-score   | ROC AUC    |
| ------------ | ---------- | ---------- | ---------- | ---------- |
| RF           | 0.2667     | 0.1905     | 0.2222     | 0.7849     |
| LR           | 0.4000     | 0.0952     | 0.1538     | 0.8500     |
| SVM          | 0.2500     | 0.0476     | 0.0800     | 0.7765     |
| GRU          | 0.9100     | 0.9220     | 0.9160     | 0.9650     |
| LSTM         | 0.8870     | 0.9220     | 0.9040     | 0.9600     |
| RETAIN       | 0.9580     | 0.8960     | 0.9260     | 0.9660     |
| RNN          | 0.9474     | 0.9351     | 0.9412     | 0.9692     |
| RNN+rev      | 0.9494     | 0.9740     | 0.9615     | 0.9860     |
| RNN+max_pool | 0.9375     | 0.9740     | 0.9554     | 0.9758     |
| RNN+rev      | 0.9351     | 0.9351     | 0.9351     | 0.9844     |
|  + max_pool  |            |            |            |            |


* All claims should be supported by experiment results
    * r


* Discuss with respect to the hypothesis and results from the original paper
    * r


* Experiments beyond the original paper Each experiment should include results and a discussion
    * r


* Ablation Study.
    * r


# Discussion
Implications of the experimental results, whether the original paper was reproducible, and if it wasn’t, what factors made it irreproducible

“What was easy”

“What was difficult”

Recommendations to the original authors or others who work in this area for improving reproducibility

  * Make assessment that the paper is reproducible or not.
    * The original paper's code is not fully reproducible due to the lack of necessary files and resources. Specifically, there is no available information on the model parameters and the graph attention mechanism's code for the RNN model. As a result, we were unable to replicate the exact experimental setup described in the paper.
  * Explain why it is not reproducible if your results are kind negative.
    * The lack of parameter information and the absence of the graph attention mechanism's code for the RNN model in the original paper's GitHub repository make it impossible to reproduce the experiments accurately. Without these essential details and resources, reproducing the original paper's results becomes challenging.
  * Describe “What was easy” and “What was difficult” during the reproduction.
    * Understanding the high-level concepts and methodologies presented in the paper was relatively straightforward. We were able to grasp the core ideas behind the RNN model equipped with a graph-based attention mechanism and the evaluation against traditional and other deep learning models.
    * The most challenging aspect of the reproduction was the lack of detailed information and resources required to implement the exact experimental setup described in the paper. Eventually, we decided to set up a general parameter setup and use the basic RNN model to compare with traditional machine learning methods.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
    * We suggest providing comprehensive documentation and code repositories that include all necessary files, parameter information, and model implementations to facilitate reproducibility. Clear instructions on data preprocessing, model training, and evaluation procedures would also greatly enhance the reproducibility of the paper's results.


    


# Public Github Repo

https://github.com/BK1147/CS598_DLH_Final_Project.git

# References

1.   C. Yin, R. Zhao, B. Qian, X. Lv and P. Zhang, "Domain Knowledge Guided Deep Learning with Electronic Health Records," 2019 IEEE International Conference on Data Mining (ICDM), Beijing, China, 2019, pp. 738-747, doi: 10.1109/ICDM.2019.00084.

